In [1]:
import torch
import numpy as np

import plotly.express as px
import pandas as pd

from model import get_model
from data import get_dataloaders

from config import batch_size, device
from loss_metrics import dice_pytorch, iou_pytorch
from crf import dense_crf, dense_crf_batch

In [2]:
device

device(type='cuda', index=0)

In [6]:
model = get_model(model_name="unet", encoder_name="resnet18", pretrained=False)
checkpoint = torch.load(
    "outputs/unet-resnet18-B32-E50-LR0.01-PTM0-BCE_Dice-20231123-192951/model.pth"
)
model.load_state_dict(checkpoint["model_state_dict"])

_, _, test_loader = get_dataloaders(batch_size)

/home/santhise001/miniforge3/envs/lgg-segmentation/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/santhise001/miniforge3/envs/lgg-segmentation/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/santhise001/miniforge3/envs/lgg-segmentation/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/santhise001/miniforge3/envs/lgg-segmentation/lib/python3.11/site-packages/sklearn/utils/valid

In [66]:
history = {"test_IoU": [], "test_dice": []}
pred_masks = []

# for i, (image, mask) in enumerate(test_loader):
# print(image.shape, mask.shape)
with torch.no_grad():
    running_IoU = 0
    running_dice = 0
    for i, data in enumerate(test_loader):
        # print(f"Processing for batch {i} / {len(test_loader)}")
        img, mask = data
        img, mask = img.to(device), mask.int().to(device)
        predictions = model(img)
        pred_masks.append(predictions)
        # print(img.cpu().numpy().astype(np.uint8).shape)
        # print(predictions.cpu().numpy().shape)
        # predictions = dense_crf_batch(
        #     img.cpu().numpy().astype(np.uint8), predictions.cpu().numpy()
        # )
        # print(np.asarray(predictions).shape)
        # predictions = torch.from_numpy(np.asarray(predictions).squeeze(3)).to("cuda")
        predictions = predictions.squeeze(1)
        dice_losses_batch = dice_pytorch(predictions, mask).cpu().numpy()
        ious_batch = iou_pytorch(predictions, mask).cpu().numpy()
        history["test_IoU"].extend(dice_losses_batch)
        history["test_dice"].extend(ious_batch)

print(f"Test IoU: {np.asarray(history['test_IoU']).mean()}")
print(f"Test DICE: {np.asarray(history['test_dice']).mean()}")

Test IoU: 0.9377061724662781
Test DICE: 0.9072995185852051


In [68]:
pred_masks[0].shape

torch.Size([32, 1, 256, 256])

In [22]:
images, masks = next(iter(test_loader))

In [42]:
px.imshow(, axis=-1))

In [34]:
px.imshow(masks[6])

In [41]:
masks[10]

torch.Size([256, 256])

In [50]:
px.imshow(np.stack([masks[10], masks[10], masks[10]]).transpose(1, 2, 0))

In [83]:
fig = px.imshow(
    np.asarray(
        [
            images[10].permute(1, 2, 0),
            np.stack([masks[10], masks[10], masks[10]]).transpose(1, 2, 0),
            images[6].permute(1, 2, 0),
            np.stack([masks[6], masks[6], masks[6]]).transpose(1, 2, 0),
        ]
    ),
    facet_col=0,
    facet_col_wrap=4,
    # width=800,
    # height=100,
)

fig.layout.annotations[0]["text"] = "Image 1"
fig.layout.annotations[1]["text"] = "Labelled Mask 1"
fig.layout.annotations[2]["text"] = "Image 2"
fig.layout.annotations[3]["text"] = "Labelled Mask 2"

fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)

fig.write_image("sample_data.png")

In [84]:
fig.show()

In [85]:
mask1 = pred_masks[0][10].detach().cpu().numpy().squeeze()

mask1 = np.stack([mask1, mask1, mask1]).transpose(1, 2, 0)

mask2 = pred_masks[0][6].detach().cpu().numpy().squeeze()

mask2 = np.stack([mask2, mask2, mask2]).transpose(1, 2, 0)

In [86]:
fig = px.imshow(
    np.asarray([images[10].permute(1, 2, 0), mask1, images[6].permute(1, 2, 0), mask2]),
    facet_col=0,
    facet_col_wrap=4,
    # width=800,
    # height=100,
)

fig.layout.annotations[0]["text"] = "Image 1"
fig.layout.annotations[1]["text"] = "Predicted Mask 1"
fig.layout.annotations[2]["text"] = "Image 2"
fig.layout.annotations[3]["text"] = "Predicted Mask 2"

fig.update_xaxes(showticklabels=False).update_yaxes(showticklabels=False)

fig.write_image("sample_predictions.png")

In [87]:
fig.show()

In [5]:
checkpoint["history"]["val_loss"]

[0.05815606263734527,
 0.035802729794029466,
 0.027985114844168647,
 0.028904004679140397,
 0.025947846320726102,
 0.02483438990883908,
 0.026594008625311367,
 0.02390472188718238,
 0.02237465422032243,
 0.022214564984127626,
 0.027680844901982,
 0.017284144644262426,
 0.022318827739711534,
 0.02280248332200414,
 0.02436402952140671,
 0.020038693223022302,
 0.02044962667307611,
 0.021052902987447837,
 0.020249723257907366]

In [65]:
model(images[10])

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [6]:
losses

NameError: name 'losses' is not defined

In [ ]:
df = pd.DataFrame(checkpoint["history"]).drop("num_epochs_trained", axis=1)

In [ ]:
px.line(df[["train_loss", "val_loss"]])

In [ ]:
results = pd.read_csv("results.csv", header=1)

In [ ]:
results

,Model,Backbone,Val IoU,Val DICE,Test IoU,Test DICE,# Params (In Millions),pretrained,loss
0,UNet,resnet18,0.893,0.924,0.943,0.917,11,True,DICE + BCE
1,UNet,resnet34,0.898,0.926,0.939,0.911,21,True,DICE + BCE
2,UNet,resnet50,0.882,0.912,0.938,0.910,23,True,DICE + BCE
3,UNet,resnet101,0.886,0.916,0.940,0.916,42,True,DICE + BCE
4,DeepLab v3+,resnet18,0.883,0.916,0.945,0.919,11,False,DICE + BCE
5,DeepLab v3+,resnet34,0.907,0.934,0.947,0.921,21,False,DICE + BCE
6,DeepLab v3+,resnet50,0.907,0.934,0.947,0.919,23,False,DICE + BCE
7,DeepLab v3+,resnet101,0.892,0.920,0.943,0.917,42,False,DICE + BCE
8,UNet,vgg11,0.899,0.926,0.949,0.920,9,False,DICE + BCE
9,UNet,vgg19,0.873,0.903,0.921,0.891,20,True,DICE + BCE


In [ ]:
losses_df = results[
    (results["pretrained"] == False) & (results["Backbone"] == "resnet18")
]

In [ ]:
losses_df

,Model,Backbone,Val IoU,Val DICE,Test IoU,Test DICE,# Params (In Millions),pretrained,loss
4,DeepLab v3+,resnet18,0.883,0.916,0.945,0.919,11,False,DICE + BCE
10,UNet,resnet18,0.899,0.927,0.951,0.924,11,False,DICE + BCE
14,UNet,resnet18,0.904,0.928,0.929,0.904,11,False,BCE
15,DeepLab v3+,resnet18,0.883,0.916,0.932,0.902,11,False,BCE
16,DeepLab v3+,resnet18,0.875,0.905,0.939,0.914,11,False,DICE
17,UNet,resnet18,0.878,0.909,0.935,0.908,11,False,DICE


In [ ]:
px.bar(losses_df, x="loss", color="Model", barmode="group", y="Test IoU")